# Set up environment and packages

In [27]:
import sys
print(sys.executable)
import mesa
import jupyter
import numpy as np
import pandas as pd
import seaborn as sns
from mesa.datacollection import DataCollector
import random
from collections import Counter

c:\Users\abigail.weeks\AppData\Local\Programs\Python\Python313\python.exe


In [28]:
print(mesa.__version__)

3.2.0


# Import Developer Information and Fuel Type Characteristics 

In [29]:
developer_states_incomplete = pd.read_excel("developer_states.xlsx")
fuel_states=pd.read_excel("fuel_states.xlsx")
developer_states_incomplete['fuel_type'] = developer_states_incomplete['fuel_type'].str.strip().str.lower()
fuel_states['fuel_type'] = fuel_states['fuel_type'].str.strip().str.lower()
developer_states=developer_states_incomplete.merge(fuel_states, on='fuel_type', how='right')
new_projects_df=developer_states
merged = developer_states_incomplete.merge(fuel_states, on='fuel_type', how='right', indicator=True)
print(merged[merged['_merge'] != 'both'])  # Shows rows not matched on both sides

    unique_id fuel_type  risk_tolerance  size developer zone  \
302       NaN       gas             NaN   NaN       NaN  NaN   

    Capacity or Energy  base_bid_price  available_hours  \
302                NaN             3.0             8000   

     peak_hour_availability  off_peak_hour_availability  ELCC  \
302             1142.857143                 6857.142857  0.74   

     LMP needed to cover levelized costs  Generation 2024 GWh  \
302                                 39.0             376249.8   

     installed capacity 2024 MW   Unnamed: 9      _merge  
302                    168654.1  3142.107401  right_only  


# Import Node Information

In [30]:
node_states = pd.read_excel("node_states.xlsx")

# Set Up Agent, Node, and Model

In [31]:
#Defining Developer Class
class DeveloperAgent(mesa.Agent):
#each of the Developer States upon Initiation
    def __init__(self, model, unique_id, fuel_type, size, risk_tolerance, developer, base_bid_price, available_hours, construction_time, study_time_at_entry, bid_price, hours_operating, revenue, online=False):
        super().__init__(model)
        self.model=model
        self.unique_id = unique_id
        self.fuel_type = fuel_type
        self.size = size
        self.risk_tolerance = risk_tolerance
        self.developer= developer
        self.base_bid_price=base_bid_price
        self.available_hours=available_hours
        self.revenue = 0
        self.past_wins = 0
        self.past_dropouts = 0
        self.internal_step_count=0
        self.hours_operating = hours_operating
        self.construction_time = 7
        self.study_time_at_entry=7
        self.online = online
        self.bid_price = 0
#Defining the Functions 
#Test function for "say hi"
    def say_hi(self):
        pass #print(f"Hi, I am agent {self.unique_id} and I run on {self.fuel_type}")
#Get summary stats of this developer's past projects.
    def get_past_developer_performance(self):
        past_developer = []
        for round_data in self.model.round_results:
            for entry in round_data:
                if entry.get("developer") == self.developer:
                    past_developer.append(entry)
        return past_developer
    def get_past_node_performance(self):
        past_node = []
        for node_summary in self.model.node_summary:
            for entry in node_summary:
                if entry.get("unique_id") == self.chosen_node:
                    past_node.append(entry)
        return past_node
#Choosing Node
    def choose_node(self):
        past_projects = self.get_past_developer_performance()

        # 1. Avoid nodes where this developer dropped out recently
        N = 3
        bad_nodes = set()
        for round_data in self.model.round_results[-N:]:
            for entry in round_data:
                if entry.get("dropped_out") and entry.get("developer") == self.developer:
                    bad_nodes.add(entry.get("chosen_node"))

        # 2. Prepare scores for ranking
        valid_nodes = [n for n in self.model.nodes if n.unique_id not in bad_nodes]

        # Gather values for ranking
        lmp_vals = [n.lmp for n in valid_nodes]
        cong_vals = [n.congestion_cost for n in valid_nodes]
        queue_vals = [n.queue_size_normalized for n in valid_nodes]

        # Normalize rankings: lower rank = better
        lmp_ranks = {n.unique_id: rank for rank, n in enumerate(sorted(valid_nodes, key=lambda x: -x.lmp))}
        cong_ranks = {n.unique_id: rank for rank, n in enumerate(sorted(valid_nodes, key=lambda x: x.congestion_cost))}
        queue_ranks = {n.unique_id: rank for rank, n in enumerate(sorted(valid_nodes, key=lambda x: x.queue_size_normalized))}

        # Optional: track past NU cost by tech from last round
        last_round = self.model.round_results[-1] if len(self.model.round_results) > 0 else []
        nu_cost_ranks = {}
        if last_round:
            # Filter last round for this tech
            tech_entries = [e for e in last_round if e.get("fuel_type") == self.fuel_type]
            nu_costs = {}
            for entry in tech_entries:
                node_id = entry.get("chosen_node")
                nu_costs[node_id] = entry.get("NU_cost", 0)

            sorted_nu = sorted(nu_costs.items(), key=lambda x: x[1])
            for i, (node_id, _) in enumerate(sorted_nu):
                nu_cost_ranks[node_id] = i

        # 3. Score all valid nodes
        scored_nodes = []

        for node in valid_nodes:
            lmp_score = lmp_ranks[node.unique_id]
            cong_score = cong_ranks[node.unique_id]
            queue_score = queue_ranks[node.unique_id]
            nu_score = nu_cost_ranks.get(node.unique_id, len(valid_nodes))  # If missing, assume worst

            # Bias by fuel type
            if self.fuel_type in ["solar", "solar_storage"]:
                fuel_bias = node.percent_solar
            elif self.fuel_type == "wind":
                fuel_bias = node.percent_wind
            else:
                fuel_bias = 0

            random_adder = random.uniform(-1, 1)

            # Composite score (lower is worse, so multiply by -1)
            score = -1 * (lmp_score + cong_score + queue_score + fuel_bias + nu_score + random_adder)

            scored_nodes.append((node, score))

        # Sort by score descending (higher score = better)
        scored_nodes.sort(key=lambda x: x[1], reverse=True)

        # Pick one randomly from the top 3
        top_n = 3
        top_choices = scored_nodes[:min(top_n, len(scored_nodes))]
        chosen_node = random.choice(top_choices)[0]

        self.chosen_node = chosen_node.unique_id



    #Dropping out (this will be called in the model itself)
    def dropout(self):
    # Only allow dropout in the first round (internal_step_count == 0)
        if self.internal_step_count == 0:
            dropout_prob = 0.8
            if random.random() < dropout_prob:
                self.dropped_out = True
                pass #print(f"Agent {self.unique_id} dropped out randomly in round 1.")
            else:
                self.dropped_out = False
        else:
            self.dropped_out = False
    def get_revenue(self):
        node = next((n for n in self.model.nodes if n.unique_id == self.chosen_node), None)
    # Only calculate revenue if project is active and past study + construction
        if getattr(self, "dropped_out", False):
            return 1  # No revenue if dropped out
        if self.internal_step_count <= (self.study_time_at_entry + self.construction_time):
            return 2 
        if self.bid_price>=node.lmp:
            pass #print(f"BID REJECTED: bid={self.bid_price}, lmp={node.lmp}")
            return 3
            
        else:
            revenue = node.lmp * self.available_hours * self.size
            return revenue

    def choose_service_type(self):
        if self.internal_step_count == 0:
            if random.random() < 0.15:
                self.service_type = 'ERIS'
            else:
                self.service_type = 'NRIS'
        else:
        # For later rounds:
            if not hasattr(self, 'service_type'):
                self.service_type = 'NRIS'

            if self.service_type == "ERIS":
            # ERIS agents can switch to NRIS (say 30% chance)
                if random.random() < 0.005:
                    self.service_type = "NRIS"
        # NRIS agents never switch to ERIS
    def step(self):
        if self.internal_step_count == 0:
            self.say_hi()
        self.choose_service_type()     
        self.internal_step_count+=1

        
        


In [ ]:
#Defining the Node CLass
class Node(mesa.Agent):
#each of the Node States upon Initiation
    def __init__(self, model, unique_id, lmp, congestion_cost, available_capacity_round_0,mw_threshold_adder_1,  nu_tier_0, nu_tier_1,
                 	nu_tier_2, nu_cost_round_0,	queue_size_normalized,	bid_price_round_0, construction_delay_factor, percent_wind, percent_solar):
        super().__init__(model)
        self.unique_id = unique_id
        self.congestion_cost = congestion_cost
        self.lmp=model.system_marginal_energy_cost[0] + self.congestion_cost
        self.available_capacity_round_0 = available_capacity_round_0
        self.mw_threshold_adder_1 = mw_threshold_adder_1
        self.nu_tier_0 = nu_tier_0
        self.nu_tier_1 = nu_tier_1
        self.nu_tier_2 = nu_tier_2
        self.nu_cost_round_0 = nu_cost_round_0
        self.queue_size_normalized = queue_size_normalized
        self.bid_price_round_0 = bid_price_round_0
        self.construction_delay_factor=construction_delay_factor
        self.percent_solar=percent_solar
        self.percent_wind=percent_wind
    #Test Function for saying hi
    def say_hi(self):
        pass #print(f"Hi, I am agent {self.unique_id}")
    def update_lmp(self):
        self.lmp =float(self.lmp[-1]) - self.congestion_cost
    def update_congestion_cost(self):
        eris_mw=sum(dev.size for dev in self.model.agents_by_type[DeveloperAgent]
            if dev.chosen_node==self.unique_id and getattr(dev,"service_type")=="ERIS")
        nris_mw=sum(dev.size for dev in self.model.agents_by_type[DeveloperAgent]
            if dev.chosen_node==self.unique_id and getattr(dev,"service_type")=="NRIS")
        self.congestion_cost=float(self.congestion_cost-(eris_mw/100))
    def step(self):
        self.update_lmp()
        self.update_congestion_cost()

In [33]:
capacity_price=100

In [34]:
class InterconnectionModel(mesa.Model):
    def __init__(self, developer_states, node_states, new_projects_df, capacity_price,
                  base_construction=1, seed=None):
        super().__init__(seed=seed)
        self.new_projects_df = new_projects_df
        self.base_construction=base_construction
        self.capacity_price = capacity_price
        self.capacity_market_prices = [capacity_price]
        self.round_results = [] 
        self.round_summary=[]
        self.nodes = []
        self.node_summary_list = []
        self.capacity_market_prices=[capacity_price]
        self.current_study_time=self.assign_study_time()
        self.total_capacity_reference=10000
        self.system_marginal_energy_cost=[1.0]
        self.projects_per_node=[]
        self.datacollector = mesa.DataCollector(
    model_reporters={
        "num_total_projects": lambda m: len(m.agents_by_type[DeveloperAgent]),
        "projects_per_node": lambda m: m.projects_per_node,
        "num_dropped_out": lambda m: sum(1 for d in m.agents_by_type[DeveloperAgent] if getattr(d, "dropped_out", False)),
        "num_online": lambda m: sum(1 for d in m.agents_by_type[DeveloperAgent] if getattr(d, "online", False)),
        "MW_online": lambda m: sum(d.size for d in m.agents_by_type[DeveloperAgent] if d.online),
        "average_revenue": lambda m: (
            sum(d.revenue for d in m.agents_by_type[DeveloperAgent]) / len(m.agents_by_type[DeveloperAgent])
        ) if len(m.agents_by_type[DeveloperAgent]) > 0 else 0,
        "capacity_price": lambda m: m.current_capacity_price,
       "ERIS_num_projects": lambda m: sum(
        1 for d in m.agents_by_type[DeveloperAgent]
        if getattr(d, "service_type", None) == "ERIS" and getattr(d, "online", False)),
        "ERIS_MW": lambda m: sum(d.size for d in m.agents_by_type[DeveloperAgent]
        if getattr(d, "service_type", None) == "ERIS" and getattr(d, "online", False)),
        "NRIS_num_projects": lambda m: sum(
        1 for d in m.agents_by_type[DeveloperAgent]
        if getattr(d, "service_type", None) == "NRIS" and getattr(d, "online", False)),
        "NRIS_MW": lambda m: sum(d.size for d in m.agents_by_type[DeveloperAgent]
        if getattr(d, "service_type", None) == "NRIS" and getattr(d, "online", False)),
        "system_marginal_energy_cost": lambda m: m.current_system_marginal_energy_cost,
        "ERIS_to_NRIS_MW_ratio": lambda m: (
        sum(d.size for d in m.agents_by_type[DeveloperAgent] if getattr(d, "service_type", None) == "ERIS") /
        max(1, sum(d.size for d in m.agents_by_type[DeveloperAgent] if getattr(d, "service_type", None) == "NRIS"))
    )
    }
)
    
        
        # ──────── Load DeveloperAgents ────────
        for i, row in developer_states.iterrows():
            hours_operating=8
            construction_time=7
            study_time_at_entry=7
            bid_price=0
            revenue=0
            agent = DeveloperAgent(
                unique_id=row['unique_id'],
                model=self,
                fuel_type=row['fuel_type'],
                size=row['size'],
                risk_tolerance=row['risk_tolerance'],
                developer=row['developer'],
                base_bid_price=row['base_bid_price'],
                hours_operating=hours_operating,
                construction_time=construction_time,
                study_time_at_entry=study_time_at_entry,
                bid_price=bid_price,
                revenue=revenue, 
                available_hours=row["available_hours"]
            )
            
        
        # ──────── Load Nodes ────────
        for i, row in node_states.iterrows():
            node = Node(
                unique_id=row['unique_id'],
                model=self,
                congestion_cost=row['congestion_cost'],
                lmp = row['lmp'],
                available_capacity_round_0=row['available_capacity_round_0'],
                mw_threshold_adder_1=row['mw_threshold_adder_1'],
                nu_tier_0=row['nu_tier_0'],
                nu_tier_1=row['nu_tier_1'],
                nu_tier_2=row['nu_tier_2'],
                nu_cost_round_0=row['nu_cost_round_0'],
                queue_size_normalized=row['queue_size_normalized'],
                bid_price_round_0=row['bid_price_round_0'],
                construction_delay_factor=row['construction_delay_factor'],
                percent_solar=row['percent_solar'],
                percent_wind=row['percent_wind']
            )
            self.nodes.append(node)
        node_ids = [node.unique_id for node in self.nodes]
       
    
    # Add new Developer Agents after round 0    
    def add_new_projects(self):
        if self.steps < 1:
            return 
        for i, row in self.new_projects_df.iterrows():
            new_unique_id = f"{row['unique_id']}_round{self.steps}"
            hours_operating=8
            construction_time=0
            study_time_at_entry=0
            bid_price=0
            revenue=0
            agent = DeveloperAgent(
                unique_id=new_unique_id,
                model=self,
                fuel_type=row['fuel_type'],
                size=row['size'],
                risk_tolerance=row['risk_tolerance'],
                developer=row['developer'],
                base_bid_price=row['base_bid_price'],
                hours_operating=hours_operating,
                construction_time=construction_time,
                study_time_at_entry=study_time_at_entry,
                bid_price=bid_price,
                revenue=revenue,
                available_hours=row['available_hours']
            )
        
    #Assign NU costs to each agent by node
    def assign_costs(self):
        for dev in [a for a in self.agents if isinstance(a, DeveloperAgent)]:
            if getattr(dev, "dropped_out", False):
                continue
            if dev.chosen_node is None:
                continue

            node = next((n for n in self.nodes if n.unique_id == dev.chosen_node), None)
            dev.assigned_cost = node.nu_cost_round_0 + 0.1 * node.queue_size_normalized * dev.size
    def calculate_node_network_upgrade_costs(self):
        node_costs = {}
        for dev in self.agents_by_type[DeveloperAgent]:
            node = dev.chosen_node  # 
        node_costs[node]['total_cost'] += dev.assigned_cost
        node_costs[node]['total_size'] += dev.size
        self.node_upgrade_costs = {}
        for node_id, vals in node_costs.items():
                avg_nu_cost_per_MW = vals['total_cost'] / vals['total_size']
        self.node_upgrade_costs[node_id] = avg_nu_cost_per_MW
    #Assign Study Time 
    def assign_study_time(self):
        if len(self.round_results) < 1:
            dropout_count = 0
        else:
            last_round = self.round_results[-1]
            dropout_count = sum(.25 for entry in last_round if entry.get("dropped_out", False))
        self.current_study_time = 1#+dropout_count
        for dev in [a for a in self.agents if isinstance(a, DeveloperAgent)]:
            if getattr(dev, "dropped_out", False):
             continue
            node = next((n for n in self.nodes if n.unique_id == dev.chosen_node), None)
            dev.study_time_at_entry = self.current_study_time
        #Assign Construction Time
    def assign_construction_time(self):
        for dev in [a for a in self.agents if isinstance(a, DeveloperAgent)]:
            if getattr(dev, "dropped_out", False):
             continue
            node = next((n for n in self.nodes if n.unique_id == dev.chosen_node), None)
            dev.construction_time = 1
            
    def assign_bid_price(self):
        for dev in [a for a in self.agents if isinstance(a, DeveloperAgent)]:
            if getattr(dev, "dropped_out", False):
                continue
            node = next((n for n in self.nodes if n.unique_id == dev.chosen_node), None)
            dev.bid_price = (
                dev.base_bid_price
                + dev.assigned_cost/1000
            )
    #Come online
    def update_online_status(self):
        for dev in self.agents_by_type[DeveloperAgent]:
            if not dev.online:
                if dev.internal_step_count >= int(dev.study_time_at_entry + dev.construction_time):
                    dev.online = True
    #Update Total Generation
    def update_total_capacity(self):
        for dev in self.agents_by_type[DeveloperAgent]:
            if dev.online and dev.internal_step_count == int(dev.study_time_at_entry + dev.construction_time) + 1:
                self.total_capacity_reference += dev.size  
    # Update Capacity Price
    def update_capacity_price(self):
        last_price = self.capacity_market_prices[-1]
        new_online_mw = 0
        for dev in self.agents_by_type[DeveloperAgent]:
         if dev.online and dev.internal_step_count == int(self.current_study_time + dev.construction_time) + 1:
            new_online_mw += dev.size
        new_price = last_price * (1 - new_online_mw/self.total_capacity_reference)
        self.capacity_market_prices.append(new_price)
        self.current_capacity_price = new_price
    def update_system_marginal_energy_cost(self):
        last_price = self.system_marginal_energy_cost[-1]
        new_online_bid_price_sum = 0.0
        new_online_count       = 0

        for dev in self.agents_by_type[DeveloperAgent]:
            if (dev.online and dev.internal_step_count== int(dev.study_time_at_entry + dev.construction_time) + 1):
             new_online_bid_price_sum += dev.bid_price
             new_online_count        += 1
        if new_online_count:
                avg_new_online_bid_price = new_online_bid_price_sum / new_online_count
        else:                                # no one came online this step
                avg_new_online_bid_price = 0.0
        new_price = last_price * (1 - avg_new_online_bid_price / self.total_capacity_reference)
        #print("  new_price:", new_price)
        self.system_marginal_energy_cost.append(new_price)
        self.current_system_marginal_energy_cost = new_price
    #build transmisison for nodes with lowest congestion adder (could also do this by zone, if I make a zone piece of the puzzle and assign nodes 
    #to it like a network proxy)
    def build_transmission (self):
          node_congestion = [(node, node.congestion_cost) for node in self.agents_by_type[Node]]
          top_nodes = sorted(node_congestion, key=lambda x: x[1], reverse=True)[:5]
          for node, _ in top_nodes:
              node.congestion_cost += 5
    
    #Define order of Functions for each Step in Model
    def step(self):
        for dev in self.agents_by_type[DeveloperAgent]:
            if dev.internal_step_count == 0:
             dev.choose_node()
        for dev in self.agents_by_type[DeveloperAgent]:
            if dev.internal_step_count == 0:
             dev.choose_service_type()
        self.assign_costs()
        for dev in self.agents_by_type[DeveloperAgent]:
            if dev.internal_step_count == 0:
             dev.dropout()
        for dev in self.agents_by_type[DeveloperAgent]:
            node_choices = [
                getattr(dev, "chosen_node", None)
        for dev in self.agents_by_type[DeveloperAgent]
        if not getattr(dev, "dropped_out", False)
        ]
            projects_this_round = dict(Counter(node_choices))
        self.projects_per_node.append(projects_this_round)
        self.assign_study_time()
        self.assign_construction_time()
        self.update_online_status()
        self.update_total_capacity()
        self.update_capacity_price()
        self.assign_bid_price()
        self.update_system_marginal_energy_cost()
        for dev in self.agents_by_type[DeveloperAgent]:
            dev.revenue = dev.get_revenue()
        for node in self.nodes:
            node.step()
        
        round_data = []
        
        for dev in self.agents_by_type[DeveloperAgent]:
            node_obj = next((n for n in self.nodes if n.unique_id == dev.chosen_node), None)
            round_data.append({
            "dev_id": dev.unique_id,
            "step": dev.internal_step_count,
            "chosen_node": int(getattr(dev, "chosen_node", None)),
            "revenue": dev.revenue,
            "dropped_out": getattr(dev, "dropped_out", False),
            "developer": dev.developer,
            "assigned_cost": dev.assigned_cost,
            "assigned_study_time": dev.study_time_at_entry,
            "assigned_construction_time": getattr(dev, "construction_time", None),
            "online": dev.online,
            "Model Step":self.steps,
            "Capacity Price":self.current_capacity_price,
            "Service_Type": dev.service_type,
            "Bid Price": dev.bid_price,
            "Revenue":dev.revenue
        })
        self.round_results.append(round_data)
        self.datacollector.collect(self)
          # ── 4. model‑level summary stats for this round ──────────────────────────
        num_total   = len(self.agents_by_type[DeveloperAgent])
        num_dropped = sum(1 for d in self.agents_by_type[DeveloperAgent] if getattr(d, "dropped_out", False))
        num_online = sum(1 for d in self.agents_by_type[DeveloperAgent] if getattr(d, "online", False))
        avg_revenue = (sum(d.revenue for d in self.agents_by_type[DeveloperAgent]) / num_total) if num_total else 0
        total_online_mw = sum(d.size for d in self.agents_by_type[DeveloperAgent] if d.online)
        # ERIS counts and MW
        eris_agents = [d for d in self.agents_by_type[DeveloperAgent] if getattr(d, "service_type", None) == "ERIS"]
        eris_num_projects = len(eris_agents)
        eris_mw = sum(d.size for d in eris_agents)

# NRIS counts and MW
        nris_agents = [d for d in self.agents_by_type[DeveloperAgent] if getattr(d, "service_type", None) == "NRIS"]
        nris_num_projects = len(nris_agents)
        nris_mw = sum(d.size for d in nris_agents)
        summary = {
            "model_step": self.steps,
            "num_total_projects": num_total,
            "num_dropped_out": num_dropped,
            "num_online": num_online,
            "MW_online": total_online_mw,
            "average_revenue": avg_revenue,
            "capacity_price": self.current_capacity_price,
            "ERIS_num_projects": eris_num_projects,
            "ERIS_MW": eris_mw,
            "NRIS_num_projects": nris_num_projects,
            "NRIS_MW": nris_mw,
            "system_marginal_energy_cost": self.current_system_marginal_energy_cost,
            "projects_per_node":projects_this_round
            }
        self.round_summary.append(summary)
        node_summary =[]
        for node in self.nodes:
    # Count developers assigned to this node
            num_projects = sum(1 for dev in self.agents_by_type[DeveloperAgent] if dev.chosen_node == node.unique_id)
            node_summary.append({
            "model_step": self.steps,
            "node_id": node.unique_id,
            "lmp": node.lmp,
            "available_capacity_round_0": node.available_capacity_round_0,
            "queue_size_normalized": node.queue_size_normalized,
            "num_projects": num_projects,
            "congestion_cost": node.congestion_cost
        })
        self.node_summary_list.append(node_summary)
        self.build_transmission()
            # Remove dropped out developers
        for dev in list(self.agents_by_type[DeveloperAgent]):
            if getattr(dev, "dropped_out", False):
                dev.remove()
        self.agents.shuffle_do("step")
        self.add_new_projects()


    

# Running the Model

In [35]:
model_instance = InterconnectionModel( developer_states, node_states, new_projects_df,capacity_price)
for _ in range(50):
    model_instance.step()
print("Final projects per node:", model_instance.projects_per_node)
projects_list = [round_data.get("projects_per_node", {}) for round_data in model_instance.round_summary]

# Create DataFrame where each row is a step and columns are nodes
projects_df = pd.DataFrame(projects_list).fillna(0).astype(int)

print(projects_df)
    
# # Use your model instance name
# flat_results = [entry for round_data in model_instance.round_results for entry in round_data]


# # Convert to DataFrame
# df = pd.DataFrame(flat_results)

# # Save to CSV
# df.to_csv("developer_round_results.csv", index=False)
# df2 = pd.DataFrame(model_instance.round_summary)  # Make sure you pass the list of dicts here

# df2.to_csv('model_round_summary.csv', index=False)
# for start in range(0, len(df2), 50):
#     print(df2.iloc[start:start+50])
# print("Steps taken:", model_instance.steps)
# print("Length of round_summary:", len(model_instance.round_summary))

all_node_rows = [row for step_list in model_instance.node_summary_list for row in step_list]
node_summary_df = pd.DataFrame(all_node_rows)
node_summary_df.to_csv('node_round_summary.csv')


Final projects per node: [{3.0: 19, 20.0: 18, 16.0: 17, 11.0: 2}, {3.0: 19, 20.0: 35, 16.0: 18, 11.0: 17, 15.0: 18, 14.0: 6}, {3.0: 19, 20.0: 46, 16.0: 24, 11.0: 25, 15.0: 38, 14.0: 16}, {3.0: 19, 20.0: 46, 16.0: 35, 11.0: 34, 15.0: 55, 14.0: 34, 1.0: 8, 17.0: 3, 12.0: 3}, {3.0: 19, 20.0: 46, 16.0: 49, 11.0: 45, 15.0: 61, 14.0: 45, 1.0: 11, 17.0: 19, 12.0: 9}, {3.0: 19, 20.0: 46, 16.0: 54, 11.0: 59, 15.0: 74, 14.0: 52, 1.0: 16, 17.0: 30, 12.0: 9, 8.0: 1}, {3.0: 19, 20.0: 46, 16.0: 60, 11.0: 74, 15.0: 87, 14.0: 58, 1.0: 19, 17.0: 38, 12.0: 12, 8.0: 1}, {3.0: 19, 20.0: 46, 16.0: 64, 11.0: 83, 15.0: 87, 14.0: 73, 1.0: 23, 17.0: 48, 12.0: 12, 8.0: 15}, {3.0: 33, 20.0: 46, 16.0: 74, 11.0: 89, 15.0: 87, 14.0: 75, 1.0: 32, 17.0: 48, 12.0: 12, 8.0: 19, 5.0: 1}, {3.0: 50, 20.0: 46, 16.0: 81, 11.0: 99, 15.0: 87, 14.0: 84, 1.0: 36, 17.0: 48, 12.0: 12, 8.0: 26, 5.0: 3}, {3.0: 63, 20.0: 46, 16.0: 93, 11.0: 108, 15.0: 87, 14.0: 90, 1.0: 42, 17.0: 48, 12.0: 12, 8.0: 38, 5.0: 7}, {3.0: 69, 20.0: 46, 1

PermissionError: [Errno 13] Permission denied: 'node_round_summary.csv'

In [37]:
node_summary_df.to_csv('node_round_summary.csv')

In [36]:
from mesa.visualization import SolaraViz, make_plot_component
import solara
model_params = {
    "capacity_price": capacity_price,
    "developer_states": developer_states,
    "node_states": node_states,
    "new_projects_df": new_projects_df,
    "base_construction": 1,
    "seed": None
}

# Your agent portrayal function
def agent_portrayal(agent):
    color = "gray"
    if hasattr(agent, "fuel_type"):
        if agent.fuel_type == "solar":
            color = "yellow"
        elif agent.fuel_type == "wind":
            color = "green"
        elif agent.fuel_type == "natural_gas":
            color = "red"

    label = getattr(agent, "chosen_node", "None")

    return {
        "color": color,
        "size": 50,
        "label": f"{agent.fuel_type}\n{label}"
    }

# Now define the visualization using your model class (not instance)
page = SolaraViz(
    model_instance,  # <-- Pass class, not instance
    [
        make_plot_component(["NRIS_num_projects", "ERIS_num_projects", "num_online"]),
        make_plot_component(["NRIS_MW", "ERIS_MW"]),
        make_plot_component("system_marginal_energy_cost"),
        #make_plot_component("average_revenue"),
        #make_plot_component("num_total_projects"),
        make_plot_component("num_dropped_out"),
        #make_plot_component("num_online"),
       #make_plot_component("MW_online"),
        make_plot_component ("capacity_price"),
        #make_plot_component ("projects_per_node"),
        #make_plot_component("ERIS_to_NRIS_MW_ratio")
    ],
    model_params=model_params
)


page



Cannot show ipywidgets in text